In [143]:
import pandas as pd
import spacy

In [144]:
nlp = spacy.load("en_core_web_sm")

In [145]:
#Using Spacy(Open source NLP library) to compare student answers with memo answers
def spacy_similarity(Answer_student,answer_memo,threshold=0.6
):
   student_response = nlp(Answer_student)
   memo_response = nlp(answer_memo)
   similarity = student_response.similarity(memo_response)
   return 2 if similarity >= threshold else 0 

In [146]:
#Read the Excel file that has student Answers
Studentscript = pd.read_excel("StudentScript.xlsx", sheet_name="Sheet1")

#Read the Excel file that has the correct Answers
Memo = pd.read_excel("Memo.xlsx", sheet_name="Sheet1")

In [147]:
#Merge the two dataframes
Compare = pd.merge(Studentscript, Memo, on="QuestionNumber", suffixes=('_Student', '_Memo'))
#Compare['Mark']= Compare.apply(lambda row: 2 if row['Answers_Student'] == row['Answers_Memo'] else 0, axis=1)

In [148]:
#For the semantic evaluation using spacy(An open-sourse NLP library)
#The evaluation kicks in if the response has greater than 7 characters
def smart_marking(row):
    if len(str(row['Answers_Student'])) > 7:
        return spacy_similarity(row['Answers_Student'], row['Answers_Memo'])
    else:
        return 2 if row['Answers_Student'] == row['Answers_Memo'] else 0

In [149]:
#Compare['Mark']= Compare.apply(lambda row: 2 if row['Answers_Student'] == row['Answers_Memo'] else 0, axis=1)
Compare['Mark'] = Compare.apply(smart_marking, axis=1)
Compare

C:\Users\rafey\AppData\Local\Temp\ipykernel_26884\1076004353.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = student_response.similarity(memo_response)


,QuestionNumber,Answers_Student,Answers_Memo,Mark
0,1,A,A,2
1,2,B,B,2
2,3,Learn,Telkom,0
3,4,TUT,Marking App,0
4,5,The capital of France is Paris,Paris is the capital city of France.,2
5,6,We will win the competition,The competition will be won by us.,0
6,7,The Capital city of South Africa is Pretoria,Pretoria,0


In [150]:
#Save the results to an Excel file
Compare.to_excel("Marking.xlsx", index=False)